<a href="https://colab.research.google.com/github/CosmicSelection/PLC-Test-2-Lexical-and-Syntax-Analyzer/blob/main/LexicalAndSyntaxAnalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#List of Lexemes to Compare to code file
lexicon = {
   '~' : '~',
   ';' : ';',
   ':' : ':',
   '+' : '+',
   '-' : '-',
   '*' : '*',
   '/' : '/', 
   '%' : '%',
   '=' : '=',
   '==': '==',
   '<' : '<', 
   '>' : '>',
   '<=': '<=',
   '>=': '>=',
   '(' : '(',
   ')' : ')',
   'loop': 'loop',
   'option': 'selection',
   'other': 'Other',
   'exit' : 'exit',
   'type': ['b2', 'b4', 'b8'],
   'letters': ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j',
               'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
               'u', 'v', 'w', 'x', 'y', 'z', '_'],
}



In [110]:
import sys

#Converts Code to a List of Lexemes and Returns Error if Something Can't be Processed
def lexicalAnalyzer(lexicon, words):
  #list for lexeme tokens
  lAnalyzed=[]

  #helps keep track of when a variable is first used
  parsedWords={}

  for lexeme in words:
    lexeme=lexeme.lower()

    #checks for a simple lexeme
    if (lexeme in lexicon.keys()):
      lAnalyzed.append({lexicon.get(lexeme)})

    #checks for a simple number value 
    elif lexeme[0].isnumeric():
      if lexeme[len(lexeme)-1]=='b' and lexeme[:len(lexeme)-1].isnumeric():
        lAnalyzed.append('value')
      elif lexeme[len(lexeme)-2:] in lexicon.get('type'): 
        lAnalyzed.append('value')
    
      #if number value doesn't appear to be simple lexeme
      #try to split number value into various lexemes (ex. 2*x is 2 * x)
      else: 
        j=0
        for lex in range(1, len(lexeme)):
          if lexeme[lex-1].isnumeric() and lexeme[lex]=='b':
            if not lexeme[j:lex].isnumeric():
              sys.exit('lexical error: '+lexeme[j: lex+1])
            lAnalyzed.append('value')
            if lexeme[lex: lex+2] in lexicon.get('type'):
              j=lex+2
              lex=lex+1
            else:
              j=lex+1
          elif lexeme[lex] in lexicon.keys():
            if lexeme[lex-1].isnumeric() and not lexeme[lex-2]=='b':
              sys.exit('lexical error: '+lexeme[j-2: lex+1])
            elif lexeme[lex:lex+2] in lexicon.keys():
              lAnalyzed.append(lexicon.get(lexeme[lex:lex+2]))
              j=lex+2
              lex=lex+1
            else:
              lAnalyzed.append(lexicon.get(lexeme[lex]))
              j=lex+1
          elif lexeme[lex] in lexicon.get('letters') and lexeme[lex+1] in lexicon.keys():
            if lex-j > 4 and lex-j < 8:
              lAnalyzed.append("variable")
            else:
              sys.exit("lexical error: "+lexeme[j:lex+1])
            if lexeme[j:lex+1] not in parsedWords.keys():
              parsedWords.update({lexeme[j:lex+1] : len(lAnalyzed)-1})
            j=lex+1
          elif lexeme[lex] in lexicon.get('letters') and not ( lexeme[lex-1] in lexicon.get('letters') or lexeme[lex-1] in lexicon.keys() ):
              sys.exit('lexical error: '+lexeme[j: lex+1])
          elif lexeme[lex] not in lexicon.get('letters') and not lexeme[lex].isnumeric():
              sys.exit('lexical error: '+lexeme[j: lex+1])


    #handles variables and combinations starting with variables such as "tuvwxyz*2b"  
    elif lexeme[0] in lexicon.get('letters'):
      j=0
      for lex in range(1, len(lexeme)):
        if lexeme[lex] in lexicon.keys():
          if lexeme[j] in lexicon.get('letters') and (lex-1-j<8 and lex-1-j>4):
            lAnalyzed.append('variable')
            if lexeme[j:lex] not in parsedWords.keys():
              parsedWords.update({lexeme[j:lex] : len(lAnalyzed)-1})
          if lexeme[lex-1].isnumeric() and not lexeme[lex-2]=='b':
            sys.exit('lexical error: '+lexeme[j-1: lex+1])
          elif lexeme[lex:lex+2] in lexicon.keys(): 
            lAnalyzed.append(lexicon.get(lexeme[lex:lex+2]))
            j=lex+2
            lex=lex+1
          elif lexeme[j] in lexicon.get('letters') and not (lex-1-j<8 and lex-1-j>4):
            sys.exit("Lexical Error: "+lexeme[j: lex]) 
          else:
            lAnalyzed.append(lexicon.get(lexeme[lex]))
            j=lex+1  
        elif lexeme[lex-1].isnumeric() and lexeme[lex]=='b':
          if not lexeme[j:lex].isnumeric():
              sys.exit('lexical error: '+lexeme[j: lex+1])
          lAnalyzed.append('value')
          if lexeme[lex:lex+2] in lexicon.get('type'):
            j=lex+2
            lex=lex+1
          else:
            j=lex+1
        elif lexeme[lex] in lexicon.get('letters') and not ( lexeme[lex-1] in lexicon.get('letters') or lexeme[lex-1] in lexicon.keys()):
              sys.exit('lexical error: '+lexeme[j: lex+1]) 
        elif lex==len(lexeme)-1:
          if lexeme[lex] not in lexicon.get('letters') or not (lex-j > 4 and lex-j <8):
              sys.exit('lexical error: '+lexeme[j: lex+1])
          lAnalyzed.append('variable')
          if lexeme[j:lex+1] not in parsedWords.keys():
              parsedWords.update({lexeme[j:lex+1] : len(lAnalyzed)-1})

        
          

    else: 
      sys.exit('lexical error: '+lexeme)
  print("Lexical Test Passed")
  return lAnalyzed, parsedWords

        


    

              


In [130]:
#Ensures Lexemes are In the Proper Order
def syntaxAnalyzer(lAnalyzed, parsedWords):

  #Ensures the code atleast starts and ends properly
  if '~' not in lAnalyzed[0] and lAnalyzed[len(lAnalyzed)-1]:
    sys.exit("CodeBlock Has Not Been Opened and/or Closed")

  #loops through all lexemes to ensure theyre initialized properly
  else:
    #stacks to check for nesting

    #selections opened and closed checker
    op=[]

    #loops opened and closed checker
    lp=[]
    for i in range(1, len(lAnalyzed)):

      #Uses parsedWords to know when a variable first appears and makes sure its initialized properly
      if 'variable' in lAnalyzed[i] and i in parsedWords.values():
        if '=' in lAnalyzed[i+1] and (('value' in lAnalyzed[i+2] or 'variable' in lAnalyzed[i+2]) and i+2 not in parsedWords.values()) and ';' in lAnalyzed[i+3]:
          i=i+3
        else:
          sys.exit("Syntax Error: Variable Improperly Initialized")

      #if its not the first instance of a variable make sure it follows the rules for an equation
      elif 'variable' in lAnalyzed[i] and '=' in lAnalyzed[i+1] and ('variable' in lAnalyzed[i+2] or 'value' in lAnalyzed[i+2] ):
        i=i+3
        p=[]
        for j in range(i, len(lAnalyzed)):
          i=j
          if ('variable' in lAnalyzed[j-1] or 'value' in lAnalyzed[j-1] or ')' in lAnalyzed[j]) and ';' in lAnalyzed[j]:
            break;
          elif '(' in lAnalyzed[j] and ('value' in lAnalyzed[j+1] or 'variable' in lAnalyzed[j+1] or '(' in lAnalyzed[j+1]):
            p.append['(']
          elif ')' in lAnalyzed[j] and len(p) > 0 and ('value' in lAnalyzed[j-1] or 'variable' in lAnalyzed[j-1] or ')' in lAnalyzed[j-1]):
            p.pop()
          elif ')' in lAnalyzed[j] :
            sys.exit("Syntax Error: Improper equation")
          elif '(' in lAnalyzed[j] :
            sys.exit("Syntax Error: Improper equation")
          elif ';' in lAnalyzed[j]:
            sys.exit("Syntax Error: Improper equation")
          elif ('variable' in lAnalyzed[j-1] or 'value' in lAnalyzed[j-1]) and not ('*' in lAnalyzed[j] or '+' in lAnalyzed[j] or '-' in lAnalyzed[j] or '/' in lAnalyzed[j] or '%' in lAnalyzed[j]):
            sys.exit("Syntax Error: Improper equation")
          elif ('*' in lAnalyzed[j-1] or '+' in lAnalyzed[j-1] or '-' in lAnalyzed[j-1] or '/' in lAnalyzed[j-1] or '%' in lAnalyzed[j-1] or '(' in lAnalyzed[j-1]) and not('variable' in lAnalyzed[j] or 'value' in lAnalyzed[j]):
            sys.exit("Syntax Error: Improper equation")
          elif j==len(lAnalyzed)-1:
            sys.exit("Syntax Error: Equation Never Ends")

      #checks to see if selection statements follow opening syntax
      #pushes opening value(s) to selection stack
      elif 'selection' in lAnalyzed[i]:
        op.append(1)
      
        if 'other' in lAnalyzed[i+1] and ':' in lAnalyzed[i+2]:
          op[len(op)-1]=3 
        elif 'value' in lAnalyzed[i+1] or 'variable' in lAnalyzed[i+1] or '(' in lAnalyzed[i+1]:
          p=[]
          if '(' in lAnalyzed[i+1]:
            p.append('(') 
          i=i+2
          for j in range(i, len(lAnalyzed)):
            i=j
            if ('<' in lAnalyzed[j] or '>' in lAnalyzed[j] or '<=' in lAnalyzed[j] or '>=' in lAnalyzed[j] or '==' in lAnalyzed[j]) and ('value' in lAnalyzed[j-1] or 'variable' in lAnalyzed[j-1] or ')' in lAnalyzed[j-1]) and op[len(op)-1]==1:
              op[len(op)-1]=2
            elif '(' in lAnalyzed[j] and ('value' in lAnalyzed[j+1] or 'variable' in lAnalyzed[j-1]):
              p.append['(']
            elif ')' in lAnalyzed[j] and len(p) > 0 and ('value' in lAnalyzed[j-1] or 'variable' in lAnalyzed[j-1] or '(' in lAnalyzed[j-1]):
              p.pop()
            elif ')' in lAnalyzed[j]:
              sys.exit("Syntax Error: Option(s) Not Initialized Properly")
            elif '(' in lAnalyzed[j]:
              sys.exit("Syntax Error: Option(s) Not Initialized Properly")
            elif ':' in lAnalyzed[j] and ('value' in lAnalyzed[j-1] or 'variable' in lAnalyzed[j-1] or ')' in lAnalyzed[j-1]) and op[len(op)-1]==2:
              op[len(op)-1]=3
              break
            elif ('+' in lAnalyzed[j] or '-' in lAnalyzed[j] or '*' in lAnalyzed[j] or '/' in lAnalyzed[j] or '%' in lAnalyzed[j]) and ('value' in lAnalyzed[j-1] or 'variable' in lAnalyzed[j-1] or ')' in lAnalyzed[j-1]):
              pass
            elif ('value' in lAnalyzed[j] or 'variable' in lAnalyzed[j]) and ('<' in lAnalyzed[j-1] or '>' in lAnalyzed[j-1] or '<=' in lAnalyzed[j-1] or '>=' in lAnalyzed[j-1] or '==' in lAnalyzed[j-1] or '+' in lAnalyzed[j-1] or '-' in lAnalyzed[j-1] or '*' in lAnalyzed[j-1] or '/' in lAnalyzed[j-1] or '%' in lAnalyzed[j-1] or '(' in lAnalyzed[j-1] or ')' in lAnalyzed[j-1]):
              pass
            elif j==len(lAnalyzed)-1:
              sys.exit("Syntax Error: Option(s) Not Initialized Properly")
            else:
              sys.exit("Syntax Error: Option(s) Not Initialized Properly")
        else:
          sys.exit("Syntax Error: Option(s) Not Initialized Properly")
        
      #pops from selection stack if it finds a hanging end of line character  
      elif len(op)>0 and ';' in lAnalyzed[i] and op[len(op)-1]==3:
        op.pop(len(op)-1)

      #Makes sure loop is initialized properly
      #pushes opening value(s) to loop stack
      elif 'loop' in lAnalyzed[i]:
        lp.append(1)
        if 'value' in lAnalyzed[i+1] or 'variable' in lAnalyzed[i+1] or '(' in lAnalyzed[i+1]:
          p=[]
          if '(' in lAnalyzed[i+1]:
            p.append('(')
          i=i+2
          for j in range(i, len(lAnalyzed)):
            i=j
            if ('<' in lAnalyzed[j] or '>' in lAnalyzed[j] or '<=' in lAnalyzed[j] or '>=' in lAnalyzed[j] or '==' in lAnalyzed[j]) and ('value' in lAnalyzed[j-1] or 'variable' in lAnalyzed[j-1] or ')' in lAnalyzed[j-1]) and lp[len(lp)-1]==1:
              lp[len(lp)-1]=2
            elif '(' in lAnalyzed[j] and ('value' in lAnalyzed[j+1] or 'variable' in lAnalyzed[j-1]):
              p.append['(']
            elif ')' in lAnalyzed[j] and len(p) > 0 and ('value' in lAnalyzed[j-1] or 'variable' in lAnalyzed[j-1] or '(' in lAnalyzed[j-1]):
              p.pop()
            elif ')' in lAnalyzed[j]:
              sys.exit("Syntax Error: Loop Not Initialized Properly")
            elif '(' in lAnalyzed[j]:
              sys.exit("Syntax Error: Loop Not Initialized Properly")
            elif ':' in lAnalyzed[j] and ('value' in lAnalyzed[j-1] or 'variable' in lAnalyzed[j-1] or ')' in lAnalyzed[j-1]) and lp[len(lp)-1]==2:
              lp[len(lp)-1]=3
              break
            elif ('+' in lAnalyzed[j] or '-' in lAnalyzed[j] or '*' in lAnalyzed[j] or '/' in lAnalyzed[j] or '%' in lAnalyzed[j]) and ('value' in lAnalyzed[j-1] or 'variable' in lAnalyzed[j-1] or ')' in lAnalyzed[j-1]):
              pass
            elif ('value' in lAnalyzed[j] or 'variable' in lAnalyzed[j]) and ('<' in lAnalyzed[j-1] or '>' in lAnalyzed[j-1] or '<=' in lAnalyzed[j-1] or '>=' in lAnalyzed[j-1] or '==' in lAnalyzed[j-1] or '+' in lAnalyzed[j-1] or '-' in lAnalyzed[j-1] or '*' in lAnalyzed[j-1] or '/' in lAnalyzed[j-1] or '%' in lAnalyzed[j-1] or '(' in lAnalyzed[j-1] or ')' in lAnalyzed[j-1]):
              pass
            elif j==len(lAnalyzed)-1:
              sys.exit("Syntax Error: Loop Not Initialized Properly")
            else:
              sys.exit("Syntax Error: Loop Not Initialized Properly")
        else:
          sys.exit("Syntax Error: Loop Not Initialized Properly")

      #makes sure loops exit condition is stated properly
      #pops from loop stack
      elif 'exit' in lAnalyzed[i]:
        if 'value' in lAnalyzed[i+1] or 'variable' in lAnalyzed[i+1] or '(' in lAnalyzed[i+1]:
          p=[]
          if '(' in lAnalyzed[i+1]:
            p.append('(')
          i=i+2
          n=10
          for j in range(i, len(lAnalyzed)):
            i=j
            if ('<' in lAnalyzed[j] or '>' in lAnalyzed[j] or '<=' in lAnalyzed[j] or '>=' in lAnalyzed[j] or '==' in lAnalyzed[j] or '=' in lAnalyzed[j]) and ('value' in lAnalyzed[j-1] or 'variable' in lAnalyzed[j-1] or ')' in lAnalyzed[j-1]) and lp[len(lp)-1]<4:
              lp[len(lp)-1]=4
            elif '(' in lAnalyzed[j] and ('value' in lAnalyzed[j+1] or 'variable' in lAnalyzed[j+1] or '(' in lAnalyzed[j+1]):
              lp[len(lp)-1]=lp[len(lp)-1]+1/(n*10)
              p.append['(']
            elif ')' in lAnalyzed[j] and len(p) > 0 and ('value' in lAnalyzed[j-1] or 'variable' in lAnalyzed[j-1] or '(' in lAnalyzed[j-1]):
              lp[len(lp)-1]=lp[len(lp)-1]+1/(n*10)
              p.pop()
            elif ')' in lAnalyzed[j]:
              sys.exit("Syntax Error: Exit Case Not Stated Properly")
            elif '(' in lAnalyzed[j]:
              sys.exit("Syntax Error: Exit Case Not Stated Properly")
            elif ';' in lAnalyzed[j] and ('value' in lAnalyzed[j-1] or 'variable' in lAnalyzed[j-1] or ')' in lAnalyzed[j-1]) and lp[len(lp)-1]>3:
              lp.pop(len(lp)-1)
              break
            elif ('+' in lAnalyzed[j] or '-' in lAnalyzed[j] or '*' in lAnalyzed[j] or '/' in lAnalyzed[j] or '%' in lAnalyzed[j]) and ('value' in lAnalyzed[j-1] or 'variable' in lAnalyzed[j-1] or ')' in lAnalyzed[j-1]):
              lp[len(lp)-1]=lp[len(lp)-1]+1/(n*10)
            elif ('value' in lAnalyzed[j] or 'variable' in lAnalyzed[j]) and ('<' in lAnalyzed[j-1] or '>' in lAnalyzed[j-1] or '<=' in lAnalyzed[j-1] or '>=' in lAnalyzed[j-1] or '==' in lAnalyzed[j-1] or '+' in lAnalyzed[j-1] or '-' in lAnalyzed[j-1] or ')' in lAnalyzed[j-1] or '(' in lAnalyzed[j-1]or '*' in lAnalyzed[j-1] or '/' in lAnalyzed[j-1] or '%' in lAnalyzed[j-1] or '=' in lAnalyzed[j-1]):
              pass
            elif j==len(lAnalyzed)-1:
              sys.exit("Syntax Error: Exit Case Not Stated Properly")
            else:
              sys.exit("Syntax Error: Exit Case Not Stated Properly")    
        else:
          sys.exit("Exit Case Not Stated Properly")
      
      #ensures values only appear when they're suppose to
      elif 'value' in lAnalyzed[i] and not ('variable' or '=' or 'loop' or 'exit' or 'selection' in lAnalyzed[i+1]):
        sys.exit("Syntax Error: Value Misplaced")

    print("Syntax Test Passed") 


            


          
                                                                          
        

In [135]:
file = open("NoErrorsTestCase2.txt", "rt")
data = file.read()
words = data.split()
syntaxInput=lexicalAnalyzer(lexicon, words)
lAnalyzed=syntaxInput[0]
parsedWords=syntaxInput[1]
syntaxAnalyzer(lAnalyzed, parsedWords)


Lexical Test Passed
Syntax Test Passed
